In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from datetime import datetime
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv('/content/diamonds2.csv')



In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  int64  
 2   color    53940 non-null  int64  
 3   clarity  53940 non-null  int64  
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   x        53940 non-null  float64
 7   y        53940 non-null  float64
 8   z        53940 non-null  float64
 9   price    53940 non-null  int64  
dtypes: float64(6), int64(4)
memory usage: 4.1 MB


In [4]:

# Definir las dimensiones de entrada (D_in), capa oculta (H) y dimensiones de salida (D_out)
D_in, H, D_out = 9, 150, 1  # D_out es 1 para regresión

In [5]:
# Crear el modelo de regresión
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)


In [6]:
# Convertir el modelo a CUDA si estás utilizando GPU
model.to("cuda")


Sequential(
  (0): Linear(in_features=9, out_features=150, bias=True)
  (1): ReLU()
  (2): Linear(in_features=150, out_features=1, bias=True)
)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  int64  
 2   color    53940 non-null  int64  
 3   clarity  53940 non-null  int64  
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   x        53940 non-null  float64
 7   y        53940 non-null  float64
 8   z        53940 non-null  float64
 9   price    53940 non-null  int64  
dtypes: float64(6), int64(4)
memory usage: 4.1 MB


In [8]:
# Separar características (X) y etiquetas (y)
X = data.iloc[:, :9].values
y = data.iloc[:, 9].values

In [9]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [11]:
# Convertir datos a tensores de PyTorch y copiar en GPU
X_train_tensor = torch.FloatTensor(X_train).cuda()
y_train_tensor = torch.FloatTensor(y_train).view(-1, 1).cuda()
X_test_tensor = torch.FloatTensor(X_test).cuda()
y_test_tensor = torch.FloatTensor(y_test).view(-1, 1).cuda()

In [12]:
# Definir la función de pérdida (MSE) y el optimizador (Adam)
criterion = nn.MSELoss()                                      #Mean Squared Error
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [14]:
# Bucle de entrenamiento
num_epochs = 5000
log_each = 100
train_losses = []

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)   #La función de pérdida mide cuán diferente son las predicciones del modelo de los valores reales de entrenamiento
    loss.backward()
    optimizer.step()
    train_losses.append(loss.item())

    if (epoch + 1) % log_each == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [100/5000], Loss: 2489226.0000
Epoch [200/5000], Loss: 2374830.7500
Epoch [300/5000], Loss: 2279233.0000
Epoch [400/5000], Loss: 2198854.7500
Epoch [500/5000], Loss: 2130879.5000
Epoch [600/5000], Loss: 2072997.8750
Epoch [700/5000], Loss: 2023424.5000
Epoch [800/5000], Loss: 1980749.3750
Epoch [900/5000], Loss: 1943856.2500
Epoch [1000/5000], Loss: 1911798.2500
Epoch [1100/5000], Loss: 1883824.0000
Epoch [1200/5000], Loss: 1859289.7500
Epoch [1300/5000], Loss: 1837660.1250
Epoch [1400/5000], Loss: 1818481.6250
Epoch [1500/5000], Loss: 1801356.7500
Epoch [1600/5000], Loss: 1785958.6250
Epoch [1700/5000], Loss: 1771985.5000
Epoch [1800/5000], Loss: 1759185.1250
Epoch [1900/5000], Loss: 1747350.7500
Epoch [2000/5000], Loss: 1736269.3750
Epoch [2100/5000], Loss: 1725793.5000
Epoch [2200/5000], Loss: 1715799.2500
Epoch [2300/5000], Loss: 1706173.8750
Epoch [2400/5000], Loss: 1696832.2500
Epoch [2500/5000], Loss: 1687688.0000
Epoch [2600/5000], Loss: 1678695.0000
Epoch [2700/5000], Lo

In [17]:
# Evaluar el modelo en el conjunto de prueba
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    test_loss = criterion(predictions, y_test_tensor)
    print(f'Loss en el conjunto de prueba: {test_loss.item():.4f}')

Loss en el conjunto de prueba: 1353593.3750


In [16]:

# Calcular el Error Cuadrático Medio en el conjunto de prueba
mse = mean_squared_error(y_test, predictions.cpu().numpy())
print(f'Error Cuadrático Medio en el conjunto de prueba: {mse:.4f}')

Error Cuadrático Medio en el conjunto de prueba: 1353593.3232
